I like to work with timeseries data, they are my favorites

In [ ]:
# Loading the dataset
folder = "./data/"
df_timeseries = pd.read_csv(folder + "df_timeseries_en.tsv", sep="\t")

# Convert 'datetime' column to datetime object
df_timeseries['datetime'] = pd.to_datetime(df_timeseries['datetime'])

# Displaying the dataset
df_timeseries.head(2)

In [ ]:
# Loading the metadata mental health related dataset
df_metadata = pd.read_csv(folder + "metadata_mental_best.csv")

# Creating a new sub-dataset from the time series one with only the channel_id coming from the metadata mental health dataset :
# Extract unique channel IDs from the metadata mental health dataset
mental_health_channel_ids = df_metadata['channel_id'].unique()

# Filter the time series dataset based on channel IDs
df_timeseries_mental_health = df_timeseries[df_timeseries['channel'].isin(mental_health_channel_ids)].copy()
timeseries_mental_health_channel_ids = df_timeseries_mental_health['channel'].unique()

# Set the normal indeces (0,1,2...)
df_timeseries_mental_health.reset_index(drop=True, inplace=True) 

# Display the new dataset
df_timeseries_mental_health.head(5)

In [ ]:
# Sorting of the dataset for having each videos of one channel in order
df_timeseries_mental_health['datetime'] = pd.to_datetime(df_timeseries_mental_health['datetime'])  # Convert to datetime format

# Sort the DataFrame by "channel" and "datetime"
df_timeseries_mental_health = df_timeseries_mental_health.sort_values(by=['channel', 'datetime'])

# If you want to reset the index after sorting
df_timeseries_mental_health = df_timeseries_mental_health.reset_index(drop=True)

# Display the new dataset
df_timeseries_mental_health.head(5)

In [ ]:
# Create a new column 'relative_delta_views' and set it to zero initially
df_timeseries_mental_health['relative_delta_views'] = 0.0

# Group the DataFrame by 'channel' and calculate the relative delta views
df_timeseries_mental_health['relative_delta_views'] = df_timeseries_mental_health.groupby('channel')['delta_views'].pct_change().fillna(0)

In [ ]:
df_timeseries_mental_health['week'] = df_timeseries_mental_health['datetime'] - pd.to_timedelta(df_timeseries_mental_health['datetime'].dt.dayofweek, unit='d')

df_metadata['upload_date'] = pd.to_datetime(df_metadata['upload_date'])
df_metadata['week'] = df_metadata['upload_date'] - pd.to_timedelta(df_metadata['upload_date'].dt.dayofweek, unit='d')

df_metadata.rename(columns={'channel_id': 'channel'}, inplace=True)

In [ ]:
#make the merging
df_timeseries_mental_health = df_timeseries_mental_health.merge(df_metadata, how = 'left', indicator=True, left_on= ['week', 'channel'],right_on= ['week', 'channel'])

In [ ]:
#remove columns from merging
words_to_remove = ["channel", "week"]

# Using list comprehension
columns_to_remove = [' '.join([word for word in string.split() if word not in words_to_remove]) for string in df_metadata.columns]

df_timeseries_mental_health = df_timeseries_mental_health.drop(columns=columns_to_remove, errors='ignore')


In [ ]:
df_timeseries_mental_health['_merge'] = df_timeseries_mental_health['_merge'].replace({'left_only': 0, 'both': 1, 'right_only': 0})
df_timeseries_mental_health.rename(columns={'_merge': 'mh_video'}, inplace=True)

Try to see a few channels


In [ ]:
df_timeseries_mental_health[df_timeseries_mental_health['channel']== 'UC--24Q3_ZQeFmgJE-Um5QZQ'][df_timeseries_mental_health['mh_video']==1]

In [ ]:
views_channel_try = df_timeseries_mental_health[df_timeseries_mental_health['channel']== 'UC--24Q3_ZQeFmgJE-Um5QZQ']['delta_views'].to_numpy()
indeces = df_timeseries_mental_health[df_timeseries_mental_health['channel']== 'UC--24Q3_ZQeFmgJE-Um5QZQ'].index
plt.plot(indeces, views_channel_try)